In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import gensim
import re
import sqlite3
import pickle

C:\Users\Mudit\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
#Load data from the sqlite database file into a Pandas dataframe
cnx = sqlite3.connect('Data/gutenberg/gutenberg.sqlite3')

df = pd.read_sql_query("SELECT * FROM articles", cnx)

In [3]:
#Drop index column

df.drop(columns=['id'],inplace=True)

In [4]:
df['article'] = df['article'].map(lambda x: re.sub('\r|\n|\'','',x))
df['article'] = df['article'].map(lambda x: re.sub(r'--\d\d\d-\d\d\d-\d\d\d\d','',x))
df['article'] = df['article'].map(lambda x: re.sub('\s+',' ',x))

In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words_subset = set([x for x in stop_words if 3 <= len(x) <= 5])

In [6]:
def remove_stopwords(x): #x is the string input
    word_set = x.split(' ')
    for word in word_set:
        if word in stop_words:
            word_set.remove(word)
    return ' '.join(word_set)
            


In [7]:
# Removing stopwords
df['article'] = df['article'].map(lambda x: remove_stopwords(x))

In [25]:
#df['auth_name'].iloc[100]

'Twain, Mark'

In [9]:
corpus = str()
for row in df['article'].iteritems():
    corpus += row[1]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cvectorizer = CountVectorizer(min_df = 0.1, lowercase = False)
cvectorizer.fit(df['article'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=0.1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:

X = df['article']
y = df['auth_name']

In [12]:
#Perform train test split:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25)

In [13]:
len(cvectorizer.vocabulary_) #698

698

In [14]:
X_train = cvectorizer.transform(X_train.values)

In [15]:
X_test = cvectorizer.transform(X_test.values)

In [36]:
#Building the neural network model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [43]:
input_dimension = X_train.shape[1] #This specifies the number of neurons in the input layer, in this case 444

#Using Keras' sequential API
model = Sequential()
model.add(layers.Dense(100, input_dim=input_dimension, activation='relu'))
model.add(layers.Dense(42, activation='softmax'))

In [44]:
model.compile(loss='categorical_crossentropy', 
               optimizer= 'adam', 
            metrics=['accuracy'])

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_5 (Dense)              (None, 42)                4242      
Total params: 104,342
Trainable params: 104,342
Non-trainable params: 0
_________________________________________________________________


In [40]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(df['auth_name'].unique())

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [41]:
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [46]:
model_details = model.fit(X_train, y_train,
                     epochs=100,
                     verbose=True,
                     validation_data=(X_test, y_test),
                    batch_size=10)

Train on 1575 samples, validate on 525 samples
Epoch 1/100
1575/1575 [==============================] - 8s 5ms/step - loss: 3.5049 - acc: 0.1435 - val_loss: 2.7638 - val_acc: 0.2800
Epoch 2/100
1575/1575 [==============================] - 1s 780us/step - loss: 1.7419 - acc: 0.5632 - val_loss: 1.8537 - val_acc: 0.4857
Epoch 3/100
1575/1575 [==============================] - 1s 793us/step - loss: 0.7100 - acc: 0.8686 - val_loss: 1.4452 - val_acc: 0.6457
Epoch 4/100
1575/1575 [==============================] - 1s 790us/step - loss: 0.2749 - acc: 0.9778 - val_loss: 1.2923 - val_acc: 0.6571
Epoch 5/100
1575/1575 [==============================] - 1s 797us/step - loss: 0.1289 - acc: 0.9968 - val_loss: 1.1466 - val_acc: 0.6952
Epoch 6/100
1575/1575 [==============================] - 1s 795us/step - loss: 0.0732 - acc: 0.9975 - val_loss: 1.1119 - val_acc: 0.7105
Epoch 7/100
1575/1575 [==============================] - 1s 807us/step - loss: 0.0503 - acc: 0.9975 - val_loss: 1.0643 - val_acc: 0.7

In [48]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7162


A SINGLE HIDDEN LAYER NEURAL NETWORK ARCHITECTURE APPLIED WITH SIMPLE BAG-OF_WORDS VECTORIZATION ONLY YIELDS A 70% ACCURACY

LEARNING THE DATA WITH THE SAME NEURAL NETWORK ARCHITECTURE BUT WITH TF-IDF VECTORIZATION THIS TIME.

In [ ]:
#Taking both unigrams and bigrams

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range = (1,2),max_features = 1000)
X = vectorizer.fit_transform(corpus.split(' ')) #Tfidf Vectorizer accepts a list of strings as input

In [58]:
X = vectorizer.transform(list(df['article']))

In [59]:
X.toarray().shape

(2100, 1000)

In [69]:
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [72]:
X_train.shape

(1575, 1000)

In [75]:
input_dimension = X_train.shape[1] #This specifies the number of neurons in the input layer, in this case 444
#Using Keras' sequential API
model = Sequential()
model.add(layers.Dense(100, input_dim=input_dimension, activation='relu'))
model.add(layers.Dense(42, activation='softmax'))

In [76]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
#Fitting the data:
model_details = model.fit(X_train, y_train,
                     epochs=100,
                     verbose=True,
                     validation_data=(X_test, y_test),
                    batch_size=10)

Train on 1575 samples, validate on 525 samples
Epoch 1/100
1575/1575 [==============================] - 2s 1ms/step - loss: 3.6761 - acc: 0.1206 - val_loss: 3.5510 - val_acc: 0.2629
Epoch 2/100
1575/1575 [==============================] - 1s 627us/step - loss: 3.3379 - acc: 0.3962 - val_loss: 3.1657 - val_acc: 0.4133
Epoch 3/100
1575/1575 [==============================] - 1s 624us/step - loss: 2.8271 - acc: 0.5822 - val_loss: 2.7200 - val_acc: 0.5314
Epoch 4/100
1575/1575 [==============================] - 1s 629us/step - loss: 2.3017 - acc: 0.7054 - val_loss: 2.3377 - val_acc: 0.6152
Epoch 5/100
1575/1575 [==============================] - 1s 618us/step - loss: 1.8513 - acc: 0.7829 - val_loss: 2.0230 - val_acc: 0.6629
Epoch 6/100
1575/1575 [==============================] - 1s 624us/step - loss: 1.4714 - acc: 0.8489 - val_loss: 1.7718 - val_acc: 0.7029
Epoch 7/100
1575/1575 [==============================] - 1s 776us/step - loss: 1.1664 - acc: 0.8990 - val_loss: 1.5777 - val_acc: 0.7

In [83]:
model.evaluate(X_train,y_train)

1575/1575 [==============================] - 0s 98us/step


[0.0009236828835948097, 0.9993650793650793]

In [84]:
model.evaluate(X_test,y_test)

525/525 [==============================] - 0s 91us/step


[0.47920860758849554, 0.8647619047619047]

C:\Users\Mudit\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
